In [2]:
import cv2
import numpy as np

def yolo_to_xyxy(box, img_w, img_h):
    x_center, y_center, w, h = box

    x_center *= img_w
    y_center *= img_h
    w *= img_w
    h *= img_h

    x1 = int(x_center - w / 2)
    y1 = int(y_center - h / 2)
    x2 = int(x_center + w / 2)
    y2 = int(y_center + h / 2)

    return x1, y1, x2, y2


In [3]:
def draw_boxes(image, bboxes, labels, class_names):
    img_h, img_w = image.shape[:2]

    for box, label in zip(bboxes, labels):
        x1, y1, x2, y2 = yolo_to_xyxy(box, img_w, img_h)
        class_name = class_names[int(label)]

        # Rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), (0,255,0), 2)

        # Label
        cv2.putText(image, class_name, (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    return image


In [4]:
def test_yolo_label(image_path, label_path, class_names):
    image = cv2.imread(image_path)

    bboxes = []
    labels = []

    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            label = int(parts[0])
            x_c, y_c, w, h = map(float, parts[1:])
            bboxes.append([x_c, y_c, w, h])
            labels.append(label)

    result = draw_boxes(image.copy(), bboxes, labels, class_names)

    cv2.imshow("Label Test", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [12]:
class_names = ['Board', 'Whitefigure', 'Blackfigure', 'Emptyslot']

test_yolo_label(
    "dataset/images/board01.jpg",
    "dataset/labels/board01.txt",
    class_names
)